# Multi-class Classification with ResNet50 on CIFAR-100 Using Pytorch

In [ ]:
!pip install torch torchvision scikit-learn --quiet

### Import Required Libraries
- Import PyTorch modules, torchvision models, transforms, optimizers.
- Import additional libraries for metrics and visualization.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

### Set Device
- Set the computation device to GPU (if available) or CPU.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Define Data Transforms
- Resize 32×32 CIFAR-100 images to 224×224 to match ResNet50 input requirements.
- Use random horizontal flips and rotation for data augmentation during training.
- Normalize pixel values to have zero mean and unit variance.

In [4]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

### Load CIFAR-10 Dataset
- Download training and test datasets using `torchvision.datasets`.
- Split training data into training and validation sets.

In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

In [6]:
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

### Create DataLoaders
- Split 90% of the training set for training and 10% for validation.
- Create `DataLoader` for training, validation, and testing with appropriate batch size.


In [7]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Load Pretrained ResNet50 Model
- Use pretrained ResNet50 from `torchvision.models`.
- Freeze all layers initially for feature extraction.

In [ ]:
# Load Pretrained ResNet50
resnet50 = models.resnet50(pretrained=True)

# Freeze all layers initially
for param in resnet50.parameters():
    param.requires_grad = False

### Modify Final Fully Connected Layer
- Replace `resnet.fc` with a custom classifier:
  - `Linear → ReLU → Dropout → Linear` for 100-class output.

In [9]:
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 100)
)

model = resnet50.to(device)

<img src="https://miro.medium.com/v2/resize:fit:1400/0*9LqUp7XyEx1QNc6A.png">

<img src="https://wisdomml.in/wp-content/uploads/2023/03/resnet_bannner.png">

In [10]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Define Loss Function and Optimizer
- Use `CrossEntropyLoss` for multi-class classification.
- Use `Adam` optimizer with learning rate suited for feature extraction.


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Train the Model (Feature Extraction Phase)
- Train only the new classification head (rest of the model frozen).
- Monitor training loss.

In [12]:
for epoch in range(1):
    model.train()
    total_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.9957


### Fine-tuning Deeper Layers
- Unfreeze `layer4` and the classifier (`fc`) in ResNet50.
- Update optimizer to only tune unfrozen layers with a smaller learning rate.
- Continue training for a few more epochs to improve performance.

In [13]:
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

In [14]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

In [15]:
for epoch in range(5):
    model.train()
    total_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Fine-tune] Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

[Fine-tune] Epoch 1, Loss: 0.4422
[Fine-tune] Epoch 2, Loss: 0.3015
[Fine-tune] Epoch 3, Loss: 0.2438
[Fine-tune] Epoch 4, Loss: 0.2018
[Fine-tune] Epoch 5, Loss: 0.1727


### Evaluate the Model on Test Set
- Set model to `eval()` mode and disable gradient computation.
- Collect predicted and actual labels.

In [16]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

### Confusion Matrix
- Compute and visualize the confusion matrix to analyze true positives, false positives, etc.
- Use seaborn’s heatmap for better visualization.

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, cmap='Blues', xticklabels=False, yticklabels=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

### Classification Report
- Generate precision, recall, F1-score, and support for both classes.
- Provides deeper insights into model performance beyond accuracy.

In [18]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      1000
           1       0.95      0.96      0.96      1000
           2       0.90      0.93      0.91      1000
           3       0.87      0.82      0.84      1000
           4       0.93      0.91      0.92      1000
           5       0.86      0.90      0.88      1000
           6       0.95      0.96      0.95      1000
           7       0.94      0.94      0.94      1000
           8       0.96      0.96      0.96      1000
           9       0.96      0.94      0.95      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

